In [ ]:
import numpy as np
import random
from scipy.ndimage import gaussian_filter
import plotly.graph_objects as go
from ipywidgets import interact
import ipywidgets as widgets

# Diamond-Square с сохранением шагов
def diamond_square_steps(size, roughness):
    steps = []
    height_map = np.zeros((size, size))
    height_map[0, 0] = random.random()
    height_map[0, size - 1] = random.random()
    height_map[size - 1, 0] = random.random()
    height_map[size - 1, size - 1] = random.random()

    steps.append(height_map.copy())

    step_size = size - 1
    while step_size > 1:
        half_step = step_size // 2

        # Diamond
        for x in range(0, size - 1, step_size):
            for y in range(0, size - 1, step_size):
                center = (x + half_step, y + half_step)
                avg = (height_map[x, y] + height_map[x + step_size, y] +
                       height_map[x, y + step_size] + height_map[x + step_size, y + step_size]) / 4.0
                height_map[center] = avg + random.uniform(-roughness, roughness)

        # Square
        for x in range(0, size, half_step):
            for y in range((x + half_step) % step_size, size, step_size):
                avg = 0.0
                count = 0
                if x - half_step >= 0:
                    avg += height_map[x - half_step, y]
                    count += 1
                if x + half_step < size:
                    avg += height_map[x + half_step, y]
                    count += 1
                if y - half_step >= 0:
                    avg += height_map[x, y - half_step]
                    count += 1
                if y + half_step < size:
                    avg += height_map[x, y + half_step]
                    count += 1

                height_map[x, y] = (avg / count) + random.uniform(-roughness, roughness)

        step_size //= 2
        roughness /= 2
        steps.append(height_map.copy())

    return steps

# Параметры и генерация шагов
size = 1025
roughness = 1.0
steps = diamond_square_steps(size, roughness)

# Цветовая палитра
terrain_colorscale = [
    [0.0, "#0000FF"], [0.2, "#228B22"], [0.4, "#C2B280"],
    [0.6, "#D2B48C"], [0.8, "#A9A9A9"], [1.0, "#FFFFFF"]
]

# Сетка координат
x = np.linspace(0, 1, size)
y = np.linspace(0, 1, size)
X, Y = np.meshgrid(x, y)

# Функция отрисовки сцены
def update_plot(step=0, sigma=0.0, light_x=0.0, light_y=0.0, light_z=0.5):
    height_map = steps[step]
    if sigma > 0:
        height_map = gaussian_filter(height_map, sigma=sigma)
    
    fig = go.Figure(data=[go.Surface(
        z=height_map,
        x=X,
        y=Y,
        colorscale=terrain_colorscale,
        lighting=dict(ambient=0.5, diffuse=0.7, roughness=0.9, specular=0.5),
        lightposition=dict(x=light_x, y=light_y, z=light_z),
    )])

    fig.update_layout(
        title=f"Diamond-Square Terrain (Step {step}, Sigma={sigma})",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Height"),
        margin=dict(l=10, r=10, t=50, b=10),
        height=700
    )
    fig.show()

# Интерфейс со слайдерами
interact(
    update_plot,
    step=widgets.IntSlider(min=0, max=len(steps) - 1, step=1, description='Кадр'),
    sigma=widgets.FloatSlider(min=0.0, max=5.0, step=0.1, description='Sigma'),
    light_x=widgets.FloatSlider(min=-2.0, max=2.0, step=0.1, description='Солнце X'),
    light_y=widgets.FloatSlider(min=-2.0, max=2.0, step=0.1, description='Солнце Y'),
    light_z=widgets.FloatSlider(min=0.0, max=2.0, step=0.1, description='Солнце Z')
)


interactive(children=(IntSlider(value=0, description='Кадр', max=10), FloatSlider(value=0.0, description='Sigm…

<function __main__.update_plot(step=0, sigma=0.0, light_x=0.0, light_y=0.0, light_z=0.5)>